In [10]:
import pandas as pd


path = "results/s1k_qwq-32b_templated.csv"
df = pd.read_csv(path)
df

,index,solution,question,cot_type,source_type,metadata,reasoning,attempt,attempt_answer,matches_solution,grading_explanation,templated_response
0,1,128,"Given a rational number, write it as a fractio...",math,qq8933/AIME_1983_2024,"{'ID': '1991-5', 'Year': 1991, 'Problem Number...","Okay, so the problem is asking: For how many r...","To solve the problem, we need to determine how...","""128""",True,The student correctly identified that 20! has ...,"<|im_start|>system\nYou are Qwen, created by A..."
1,2,1. **Consider a countable subset \( S_0 \subse...,Let $ \mathcal{H}$ be an infinite-dimensiona...,math,AI-MO/NuminaMath-CoT/aops_forum,"{'source': 'aops_forum', 'messages': [{'conten...","Okay, so I have this problem about an infinite...",Given an infinite-dimensional Hilbert space \(...,"""y""",True,The student’s answer correctly identifies the ...,"<|im_start|>system\nYou are Qwen, created by A..."
2,0,1. **Rewrite the function using trigonometric ...,"Let $a,b,A,B$ be given reals. We consider th...",math,AI-MO/NuminaMath-CoT/aops_forum,"{'source': 'aops_forum', 'messages': [{'conten...","Okay, so I need to prove that if the function ...",To prove that if \( f(x) = 1 - a \cos(x) - b \...,"""a^2 + b^2 \u2264 2, A^2 + B^2 \u2264 1""",True,The student’s final answer matches the require...,"<|im_start|>system\nYou are Qwen, created by A..."
3,3,109,Find the remainder when $9 \times 99 \times 99...,math,qq8933/AIME_1983_2024,"{'ID': '2010-I-2', 'Year': 2010, 'Problem Numb...","Okay, so I need to find the remainder when the...",To find the remainder when \(9 \times 99 \time...,"""109""",True,The student correctly computed the product mod...,"<|im_start|>system\nYou are Qwen, created by A..."
4,7,181,One base of a trapezoid is $100$ units longer ...,math,qq8933/AIME_1983_2024,"{'ID': '2000-II-6', 'Year': 2000, 'Problem Num...","Okay, so I have this trapezoid problem here. L...",Given a trapezoid where one base is 100 units ...,"""181""",True,The student correctly set up the problem by le...,"<|im_start|>system\nYou are Qwen, created by A..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1. **Initial Considerations**:\n - Let \( A ...,Let $A$ be a subset of $\mathbb{Z}/n\mathbb...,math,AI-MO/NuminaMath-CoT/aops_forum,"{'source': 'aops_forum', 'messages': [{'conten...","Okay, so I need to show that for a subset A of...",To show that for a subset \( A \) of \( \mathb...,"""|f(r)| \u2265 |A|/2""",True,The student’s solution uses a Fourier-analytic...,"<|im_start|>system\nYou are Qwen, created by A..."
996,998,146,"Given a circle of radius $\sqrt{13}$ , let $A$...",math,qq8933/AIME_1983_2024,"{'ID': '2013-II-10', 'Year': 2013, 'Problem Nu...","Okay, so I have this geometry problem here. Le...",Given a circle with radius \(\sqrt{13}\) and c...,"""146""",True,The student correctly placed the circle’s cent...,"<|im_start|>system\nYou are Qwen, created by A..."
997,999,1. We start with the recursive sequence define...,"Let the sequence $a_{1}, a_{2}, \cdots$ be d...",math,AI-MO/NuminaMath-CoT/aops_forum,"{'source': 'aops_forum', 'messages': [{'conten...","Okay, so I need to find the smallest possible ...",To find the smallest possible value of \( a_1 ...,"""121""",True,The student correctly solved the recurrence by...,"<|im_start|>system\nYou are Qwen, created by A..."
998,991,7.333,"Find the arc length of the curve, where x=t, y...",math,TIGER-Lab/TheoremQA/float,"{'Answer_type': 'float', 'Picture': None}","Okay, so I need to find the arc length of the ...",To find the arc length of the curve given by t...,"""5/3""",False,The student's work correctly computed the deri...,"<|im_start|>system\nYou are Qwen, created by A..."


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-32B-Instruct")


def tokenize_text(question, reasoning, attempt):
    attempt = "Answer: " + attempt if "Answer:" not in attempt else attempt

    text = tokenizer.apply_chat_template(
        [
            {"role": "user", "content": question.strip()},
            {
                "role": "assistant",
                "content": "<|im_start|>think\n"
                + reasoning.strip()
                + "\n<|im_start|>answer\n"
                + attempt.strip(),
            },
        ],
        tokenize=False,
    )
    return text


# Apply chat template to all rows in the dataframe
df["templated_response"] = df.apply(
    lambda row: tokenize_text(
        row["question"],
        row["reasoning"] if isinstance(row["reasoning"], str) else "",
        row["attempt"] if isinstance(row["attempt"], str) else "",
    ),
    axis=1,
)

df

,index,solution,question,cot_type,source_type,metadata,reasoning,attempt,attempt_answer,matches_solution,grading_explanation,templated_response
0,1,128,"Given a rational number, write it as a fractio...",math,qq8933/AIME_1983_2024,"{'ID': '1991-5', 'Year': 1991, 'Problem Number...","Okay, so I have this problem here: I need to f...","To solve the problem, we need to determine how...","""128""",True,The student correctly determined the prime fac...,"<|im_start|>system\nYou are Qwen, created by A..."
1,3,109,Find the remainder when $9 \times 99 \times 99...,math,qq8933/AIME_1983_2024,"{'ID': '2010-I-2', 'Year': 2010, 'Problem Numb...","Okay, so I have this problem here: I need to f...",To find the remainder when \(9 \times 99 \time...,"""109""",True,The student correctly expressed each factor as...,"<|im_start|>system\nYou are Qwen, created by A..."
2,5,167.0,Compute the mean molecular speed v in the heav...,math,TIGER-Lab/TheoremQA/float,"{'Answer_type': 'float', 'Picture': None}","Okay, so I need to compute the mean molecular ...",To compute the mean molecular speed \( v \) of...,"""169 m/s""",True,The student correctly applied the formula v = ...,"<|im_start|>system\nYou are Qwen, created by A..."
3,7,181,One base of a trapezoid is $100$ units longer ...,math,qq8933/AIME_1983_2024,"{'ID': '2000-II-6', 'Year': 2000, 'Problem Num...","Okay, so I have this trapezoid problem here. L...",Let the lengths of the bases of the trapezoid ...,"""181""",True,The student correctly let the bases be a and a...,"<|im_start|>system\nYou are Qwen, created by A..."
4,8,1. **Defining the Sparse Set \( S \) and the T...,"Let's say a language $L \subseteq \{0,1\}^*$ ...",math,AI-MO/NuminaMath-CoT/aops_forum,"{'source': 'aops_forum', 'messages': [{'conten...","Okay, let me try to tackle these problems one ...",**Final Answer**\n\n1. The sparse set \( S \) ...,"[""1. S = {<x, i> : x is in S_i}, and the TM M ...",True,Part 1: The student correctly combines the k s...,"<|im_start|>system\nYou are Qwen, created by A..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,998,146,"Given a circle of radius $\sqrt{13}$ , let $A$...",math,qq8933/AIME_1983_2024,"{'ID': '2013-II-10', 'Year': 2013, 'Problem Nu...","Okay, so I have this problem here about a circ...",Given a circle with radius \(\sqrt{13}\) and c...,"""146""",True,The student's attempt correctly identifies the...,"<|im_start|>system\nYou are Qwen, created by A..."
996,996,1. We start with the given sequences:\n \[\n...,"Given positive numbers $a_1$ and $b_1$ , co...",math,AI-MO/NuminaMath-CoT/aops_forum,"{'source': 'aops_forum', 'messages': [{'conten...","Okay, so I need to prove that \( a_{25} + b_{2...","Given positive numbers \(a_1\) and \(b_1\), we...","""10\u221a2""",False,"The student’s answer concludes with 10√2, but ...","<|im_start|>system\nYou are Qwen, created by A..."
997,999,1. We start with the recursive sequence define...,"Let the sequence $a_{1}, a_{2}, \cdots$ be d...",math,AI-MO/NuminaMath-CoT/aops_forum,"{'source': 'aops_forum', 'messages': [{'conten...","Okay, so I need to find the smallest possible ...",To find the smallest possible value of \( a_1 ...,"""121""",True,The student correctly solved the recurrence by...,"<|im_start|>system\nYou are Qwen, created by A..."
998,991,7.333,"Find the arc length of the curve, where x=t, y...",math,TIGER-Lab/TheoremQA/float,"{'Answer_type': 'float', 'Picture': None}","Okay, so I need to find the arc length of the ...",To find the arc length of the curve given by t...,"""5/3""",False,The student correctly computed the derivative ...,"<|im_start|>system\nYou are Qwen, created by A..."


In [11]:
# Push the dataframe to the Hugging Face Hub in the train split
from huggingface_hub import HfApi
import os
from tempfile import NamedTemporaryFile
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Create a DatasetDict with a train split
from datasets import DatasetDict

dataset_dict = DatasetDict({"train": hf_dataset})

# Save the dataset to a temporary directory
with NamedTemporaryFile(suffix=".csv", delete=False) as temp_file:
    df.to_csv(temp_file.name, index=False)
    temp_file_path = temp_file.name

# Initialize the Hugging Face API client
api = HfApi()

# Get the API token from environment variable
token = os.environ.get("HUGGINGFACE_CLI")

# Define repository details
repo_id = "jonathanyin/s1k_qwq-32b_correct_reasoning_traces"
repo_type = "dataset"

# Push the dataset to the Hub
dataset_dict.push_to_hub(repo_id=repo_id, token=token)

# Clean up the temporary file
os.unlink(temp_file_path)

print(f"Successfully pushed dataframe to {repo_id} in the train split")


Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.95s/it]


Successfully pushed dataframe to jonathanyin/s1k_qwq-32b_correct_reasoning_traces in the train split
